In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
import pandas as pd

In [12]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [13]:
!kaggle datasets download -d suchintikasarkar/sentiment-analysis-for-mental-health

Dataset URL: https://www.kaggle.com/datasets/suchintikasarkar/sentiment-analysis-for-mental-health
License(s): DbCL-1.0
100% 11.0M/11.1M [00:00<00:00, 113MB/s]
100% 11.1M/11.1M [00:00<00:00, 113MB/s]


In [16]:
import zipfile
zip_ref = zipfile.ZipFile('/content/sentiment-analysis-for-mental-health.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [19]:
df=pd.read_csv('/content/Combined Data.csv')

In [20]:
df.head()

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety


In [23]:
df['statement'][2]

'All wrong, back off dear, forward doubt. Stay in a restless and restless place'

##LowerCase


In [27]:
df['statement'][2].lower()

'all wrong, back off dear, forward doubt. stay in a restless and restless place'

In [28]:
df['statement'].str.lower()

0                                               oh my gosh
1        trouble sleeping, confused mind, restless hear...
2        all wrong, back off dear, forward doubt. stay ...
3        i've shifted my focus to something else but i'...
4        i'm restless and restless, it's been a month n...
                               ...                        
53038    nobody takes me seriously i’ve (24m) dealt wit...
53039    selfishness  "i don't feel very good, it's lik...
53040    is there any way to sleep better? i can't slee...
53041    public speaking tips? hi, all. i have to give ...
53042    i have really bad door anxiety! it's not about...
Name: statement, Length: 53043, dtype: object

In [29]:
df['statement']=df['statement'].str.lower()

In [30]:
df.head()

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"all wrong, back off dear, forward doubt. stay ...",Anxiety
3,3,i've shifted my focus to something else but i'...,Anxiety
4,4,"i'm restless and restless, it's been a month n...",Anxiety


##Remove HTML TAGS


In [32]:
text = '<html><body><p> Movie 1</p><p> Actor - Aamir Khan</p><p> Click here to <a href="http://google.com">download</a></p></body></html>'


In [38]:
import re

def remove_html_tags(text):
  if isinstance(text, str):  # Check if the input is a string
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)
  else:
    return text  # Return the original value if it's not a string

In [39]:
remove_html_tags(text)

' Movie 1 Actor - Aamir Khan Click here to download'

In [40]:
df['statement'].apply(remove_html_tags)

0                                               oh my gosh
1        trouble sleeping, confused mind, restless hear...
2        all wrong, back off dear, forward doubt. stay ...
3        i've shifted my focus to something else but i'...
4        i'm restless and restless, it's been a month n...
                               ...                        
53038    nobody takes me seriously i’ve (24m) dealt wit...
53039    selfishness  "i don't feel very good, it's lik...
53040    is there any way to sleep better? i can't slee...
53041    public speaking tips? hi, all. i have to give ...
53042    i have really bad door anxiety! it's not about...
Name: statement, Length: 53043, dtype: object

In [41]:
df['statement']=df['statement'].apply(remove_html_tags)

In [42]:
df.head()

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"all wrong, back off dear, forward doubt. stay ...",Anxiety
3,3,i've shifted my focus to something else but i'...,Anxiety
4,4,"i'm restless and restless, it's been a month n...",Anxiety


##Remove URLs


In [43]:
text1 = 'Check out my notebook https://www.kaggle.com/code/abhayparashar31/topic-modelling-and-latent-dirichlet-allocation-lda'
text2 = 'I want to check out my notebook http://www.kaggle.com/code/abhayparashar31/topic-modelling-and-latent-dirichlet-allocation-lda'
text3 = 'Google search www.google.com'


In [44]:
def remove_urls(text):
  pattern = re.compile(r'https?://\S+|www\.\S+')
  return pattern.sub(r'',text)

In [45]:
remove_urls(text1)

'Check out my notebook '

In [46]:
remove_urls(text2)

'I want to check out my notebook '

In [48]:
remove_urls(text3)

'Google search '

##Removing Punctuation

In [49]:
  import string

In [50]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [51]:
exclude = string.punctuation

In [52]:
def remove_punc(data):
  return data.translate(str.maketrans('','',exclude))


In [53]:
data = 'Yes, I play Cricket. do you ?'

In [54]:
remove_punc(data)

'Yes I play Cricket do you '

##Chat Word Treatment (large word into short form words, e.g. lol)

In [55]:
chat_words = {
    'AFAIK':'As Far As I Know',
    'AFK':'Away From Keyboard',
    'ASAP':'As Soon As Possible',
    'ATK':'At The Keyboard',
    'ATM':'At The Moment',
    'FYI': 'For your information'
}

In [57]:
chat_words

{'AFAIK': 'As Far As I Know',
 'AFK': 'Away From Keyboard',
 'ASAP': 'As Soon As Possible',
 'ATK': 'At The Keyboard',
 'ATM': 'At The Moment',
 'FYI': 'For your information'}

In [58]:
def chat_conversion(data):
  new_text= []
  for w in data.split():
    if w.upper() in chat_words:
      new_text.append(chat_words[w.upper()])
    else:
      new_text.append(w)
  return " ".join(new_text)

In [60]:
chat_conversion("I will be there ASAP")

'I will be there As Soon As Possible'

##Spelling Correction

In [66]:
my_incorrect_text = "My namee is Hafeez, my spels aree biit weak"

In [67]:
import textblob
from textblob import TextBlob

In [68]:
spell_checker = TextBlob(my_incorrect_text)
spell_checker.correct().string

'By name is Hafeez, my spell are bit weak'

##Removing Stop Words (used in sentence but not contributed in meaning of sentence)

In [79]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [80]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [81]:
def remove_stopword(data):
  new_text = []
  for word in data.split():
    if word in stopwords.words('english'):
      new_text.append('')
    else:
      new_text.append(word)
  x = new_text[:]
  new_text.clear()
  return " ".join(x)

In [82]:
data_stopword = "my name is Hafeez this is testing of stopwords removing function"


In [83]:
remove_stopword(data_stopword)

' name  Hafeez   testing  stopwords removing function'

##Emojis Handling

In [85]:
import re
def remove_emoji(data):
  emoji_pattern = re.compile("["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
  "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'',data)



In [86]:
remove_emoji("I am Happy 😂")

'I am Happy '

In [90]:
!pip install emoji
import emoji


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 16.3 MB/s eta 0:00:00


In [91]:
print(emoji.demojize('Python is 👍'))

Python is :thumbs_up:


##Tokenization

In [95]:
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [96]:
data1= 'I am going to visit Punjab'
word_tokenize(data1)

['I', 'am', 'going', 'to', 'visit', 'Punjab']

In [97]:
data2= 'lorem Ispum is a dummy text, but i want real text for practice'
sent_tokenize(data2)

['lorem Ispum is a dummy text, but i want real text for practice']

In [99]:
data3 = 'here to help you!, mail at hafeez@com'
word_tokenize(data3)

['here', 'to', 'help', 'you', '!', ',', 'mail', 'at', 'hafeez', '@', 'com']

In [100]:
import spacy

In [101]:
nlp = spacy.load('en_core_web_sm')

In [102]:
doc1 = nlp(data1)
doc2 = nlp(data2)
doc3 = nlp(data3)

In [104]:
for token in doc1:
  print(token)

I
am
going
to
visit
Punjab


In [105]:
for token in doc2:
  print(token)

lorem
Ispum
is
a
dummy
text
,
but
i
want
real
text
for
practice


In [106]:
for token in doc3:
  print(token)

here
to
help
you
!
,
mail
at
hafeez@com


In [108]:
data4= 'I have 10$ in my bank account'
doc4 = nlp(data4)

for token in doc4:
  print(token)

I
have
10
$
in
my
bank
account


##Stemming (root word, e.g. walk, walking, walked) sometime when no word is properly fix for word so it uses a word which isn't in language

In [109]:
from nltk.stem.porter  import PorterStemmer

In [110]:
ps = PorterStemmer()
def stem_words(data):
  return " ".join([ps.stem(word) for word in data.split()])

In [112]:
sample ="walk walking walked"
stem_words(sample)

'walk walk walk'

In [113]:
sample2 = "Probably we need to hangout and go to movie together"
stem_words(sample2)

'probabl we need to hangout and go to movi togeth'

##Lemmatization (it is same as stemming, it uses the same language word for group of words or word in search of root word)

In [114]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [115]:
wordnet_lemmatizer = WordNetLemmatizer()

In [116]:
senstence  = "He was running and eating at same time. He has bad habit of swimming"
punctuations = "?:!.,;"
sentence_words = nltk.word_tokenize(senstence)
for word in sentence_words:
  if word in punctuations:
    sentence_words.remove(word)
sentence_words


['He',
 'was',
 'running',
 'and',
 'eating',
 'at',
 'same',
 'time',
 'He',
 'has',
 'bad',
 'habit',
 'of',
 'swimming']